In [0]:
#3!pip uninstall -y tensorflow-gpu==2.0.0-alpha0
#!pip install --upgrade tensorflow==1.13.1
import tensorflow as tf
!pip install glove_python
tf.enable_eager_execution()
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import time
import string

import plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.graph_objs as go
from google.colab import files
from glove import Corpus, Glove

     |████████████████████████████████| 266kB 9.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
files.upload()
lines = open('fra.txt', encoding='UTF-8').read().strip().split('\n')

Saving fra.txt to fra.txt


In [0]:
exclude = list(set(string.punctuation)) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) 
exclude.append(">>")
exclude.append("<<")

In [0]:
remove_digits
exclude.remove('-')

In [0]:
def preprocess_eng_sentence(sent):
    sent = sent.lower() # lower casing
    sent = re.sub('"', '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits)
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) 
    sent = '<start> ' + sent + ' <end>' 
    return sent

In [0]:
def preprocess_fr_sentence(sent):
    sent = re.sub("<<", '', sent) # remove the quotation marks if any
    sent = re.sub(">>", '', sent)
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [0]:
sent_pairs = []
for i in range(len(lines)):
    sent_pair = []
    eng,fr = lines[i].split('\t')
    eng = preprocess_eng_sentence(eng)
    sent_pair.append(eng)
    fr = preprocess_fr_sentence(fr)
    sent_pair.append(fr)
    sent_pairs.append(sent_pair)

In [0]:
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<anything>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
def load_dataset(pairs, num_examples):
  
    inp_lang = LanguageIndex(en for en, fr in pairs)
    targ_lang = LanguageIndex(fr for en, fr in pairs)

    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, fr in pairs]
    target_tensor = [[targ_lang.word2idx[s] for s in fr.split(' ')] for en, fr in pairs]
    
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [0]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(lines))

In [0]:
input_tensor.shape

(167130, 49)

In [0]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(133704, 133704, 33426, 33426)

In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 510
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 102
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [0]:
def LSTM(units):
  if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNLSTM(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
  else:
        return tf.keras.layers.LSTM(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')
    
  

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        self.LSTM = LSTM(self.enc_units)
        
    def call(self, x, hidden1, hidden2):
        x = self.embedding(x)
        output, state1, state2 = self.LSTM(x, initial_state = [hidden1,hidden2])        
        return output, state1, state2
    
    def initialize_hidden_state(self):
        return tf.contrib.rnn.LSTMStateTuple(tf.zeros((self.batch_sz, self.enc_units)),tf.zeros((self.batch_sz, self.enc_units)))

In [0]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
    if tf.test.is_gpu_available():
        return tf.keras.layers.CuDNNGRU(units, 
                                        return_sequences=True, 
                                        return_state=True, 
                                        recurrent_initializer='glorot_uniform')
    else:
        return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.LSTM = LSTM(self.dec_units)
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden1,hidden2, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(tf.concat([hidden1, hidden2], axis=-1), 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state1,state2 = self.LSTM(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state1,state2, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [0]:
optimizer = tf.train.AdamOptimizer()

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                  encoder=encoder,
                                 decoder=decoder)

In [0]:
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE

In [0]:
EPOCHS = 7

for epoch in range(EPOCHS):
    start = time.time()
    
    [hidden1,hidden2] = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden1,enc_hidden2 = encoder(inp, hidden1,hidden2)
            
            dec_hiddenn1,dec_hiddenn2 = enc_hidden1,enc_hidden2
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden1,dec_hidden2, _ = decoder(dec_input, dec_hiddenn1,dec_hiddenn2, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.1080
Epoch 1 Batch 100 Loss 0.1261
Epoch 1 Batch 200 Loss 0.1381
Epoch 1 Loss 0.1203
Time taken for 1 epoch 377.6391797065735 sec

Epoch 2 Batch 0 Loss 0.1105
Epoch 2 Batch 100 Loss 0.1182
Epoch 2 Batch 200 Loss 0.1280
Epoch 2 Loss 0.1185
Time taken for 1 epoch 376.0354495048523 sec

Epoch 3 Batch 0 Loss 0.1092
Epoch 3 Batch 100 Loss 0.1181
Epoch 3 Batch 200 Loss 0.1229
Epoch 3 Loss 0.1168
Time taken for 1 epoch 375.69168853759766 sec

Epoch 4 Batch 0 Loss 0.1084
Epoch 4 Batch 100 Loss 0.1162
Epoch 4 Batch 200 Loss 0.1117
Epoch 4 Loss 0.1151
Time taken for 1 epoch 374.9610435962677 sec

Epoch 5 Batch 0 Loss 0.1042
Epoch 5 Batch 100 Loss 0.1050
Epoch 5 Batch 200 Loss 0.1252
Epoch 5 Loss 0.1135
Time taken for 1 epoch 375.3789026737213 sec

Epoch 6 Batch 0 Loss 0.1081
Epoch 6 Batch 100 Loss 0.1071
Epoch 6 Batch 200 Loss 0.1072
Epoch 6 Loss 0.1119
Time taken for 1 epoch 374.41227769851685 sec

Epoch 7 Batch 0 Loss 0.1143
Epoch 7 Batch 100 Loss 0.1092
Epoch 7 Batch 20

In [0]:
 checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
def evaluate(sentence):
  
    sentence = preprocess_eng_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden1,hidden2 = [tf.zeros((1, units)),tf.zeros((1, units))]
    enc_out, enc_hidden1,enc_hidden2 = encoder(inputs, hidden1,hidden2)

    dec_hidden1,dec_hidden2 = enc_hidden1,enc_hidden2
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden1,dec_hidden2, attention_weights = decoder(dec_input,
                                                             dec_hidden1,
                                                             dec_hidden2,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        #attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence#, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence #, attention_plot

In [0]:
def translate(sentence):
    result, sentence = evaluate(sentence)
    #, attention_plot
    print('Input: %s' % (sentence))
    print('Predicted translation: <start> {}'.format(result))

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
translate(u'i want to travel abroad and meet my dream girl')

Input: <start> i want to travel abroad and meet my dream girl <end>
Predicted translation: <start> Je veux rencontrer pendant que jétais jeune fille <end> 
